# Dynamic MFA Visualisation

In [1]:
import pandas as pd
import numpy as np
from dynamic_stock_model import DynamicStockModel


from bokeh.io import show, output_notebook
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

# Building the Bokeh visualization

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, curdoc 
from bokeh.models import HoverTool
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Slider, Select, Div
from bokeh.models.callbacks import CustomJS
from bokeh.palettes import viridis, Category20 

output_notebook()

Loading BokehJS ...

In [2]:
# Importing data in a pandas dataframe
file = "Sample_Data.csv"
data = pd.read_csv(file,header=0 , sep=',')
data.fillna(0, inplace=True)
#print(data)

# Initiating parameters for the lifetime distribution 
lifetime = 15
stdev=5

def compute_model(data, lifetime, stdev, input_type='Constant'):
    """
    Given an input distribution, and a number for the mean lifetime and StDev,
    This function returns a complete DSM, with all calculations done assuming
    an inflow-driven model and a normal distribution with static lifetime
    """
    
#   Initializing the Dynamic Stock Model with the input parameters
    DSM =  DynamicStockModel(t=data.loc[:,'Time'],
                             s=data.loc[:,input_type],
                             lt={'Type': 'Normal', 
                                 'Mean': np.array([lifetime]),
                                 'StdDev': np.array([stdev]) 
                                 }
                                )
#   Running the calculations, first for the stock byt time and cohort matrix,
#   then the outputs by time and cohort matrix,
#   and finally giving the total stock and outflow by time vectors
    DSM.compute_stock_driven_model() 
    DSM.compute_outflow_total()
    DSM.compute_stock_total()
        
    return DSM

# Initializing a DSM model with the default parameters
DSM = compute_model(data, lifetime, stdev)


# These data source are just used to communicate / trigger the real callback
source_lifetime = ColumnDataSource(data=dict({'lifetime':[lifetime]}))
source_stdev = ColumnDataSource(data=dict({'stdev':[stdev]}))

selected_input = 'Constant'
plot_data = {'time': DSM.t,
         'stock': DSM.s,
         'inputs': DSM.i,
         'outputs': DSM.o,
         'stock change': DSM.i - DSM.o
         }
source = ColumnDataSource(data=plot_data)
source_sc = ColumnDataSource(data={'image': [np.flipud(DSM.s_c)]})

# Data preparatin for the bar chart of cohorts vs time
cohorts_matrix = pd.DataFrame(data=DSM.s_c,  index=DSM.t.as_matrix(), columns=[str(e) for e in plot_data["time"].as_matrix().tolist()])
cohorts_matrix.insert(0, "time",[str(e) for e in plot_data["time"].as_matrix().tolist()])

source_bar = ColumnDataSource(data=cohorts_matrix)


In [3]:
# Create the Document Application
def modify_doc(curdoc):
    
    # Real callback functions to update the values from the sliders
    def update_input(attrname, old, new):
        global source, stdev, source_sc, lifetime, selected_input, source_bar
        selected_input = new
        DSM = compute_model(data, lifetime, stdev, input_type=selected_input)
        print("Input distribution", new)
        plot_data = {'time': DSM.t,
                 'stock': DSM.s,
                 'inputs': DSM.i,
                 'outputs': DSM.o,
                 'stock change': DSM.i - DSM.o
                 }
        source.data = plot_data
        source_sc.data = {'image': [np.flipud(DSM.s_c)]}
        cohorts_matrix = pd.DataFrame(data=DSM.s_c,  index=DSM.t.as_matrix(), columns=[str(e) for e in plot_data["time"].as_matrix().tolist()])
        cohorts_matrix.insert(0, "time",[str(e) for e in plot_data["time"].as_matrix().tolist()])
        source_bar_temp = ColumnDataSource(data=cohorts_matrix) 
        #temp used only to get the right format
        source_bar.data = source_bar_temp.data 

    def update_lifetime(attrname, old, new):
        global source, stdev, lifetime, source_bar
        global source_bar
        DSM = compute_model(data, source_lifetime.data['lifetime'][0], stdev, input_type=selected_input)
        print("Lifetime", source_lifetime.data['lifetime'][0])
        plot_data = {'time': DSM.t,
             'stock': DSM.s,
             'inputs': DSM.i,
             'outputs': DSM.o,
             'stock change': DSM.i - DSM.o
         }
        source.data = plot_data
        lifetime = source_lifetime.data['lifetime'][0]
        source_sc.data = {'image': [np.flipud(DSM.s_c)]}
        cohorts_matrix = pd.DataFrame(data=DSM.s_c,  index=DSM.t.as_matrix(), columns=[str(e) for e in plot_data["time"].as_matrix().tolist()])
        cohorts_matrix.insert(0, "time",[str(e) for e in plot_data["time"].as_matrix().tolist()])
        source_bar_temp = ColumnDataSource(data=cohorts_matrix) 
        #temp used only to get the right format
        source_bar.data = source_bar_temp.data 

    def update_stdev(attrname, old, new):
        global source, stdev, source_sc, lifetime, source_bar, cohorts_matrix
        DSM = compute_model(data, lifetime, source_stdev.data['stdev'][0], input_type=selected_input)
        print("StDev", source_stdev.data['stdev'][0])
        plot_data = {'time': DSM.t,
                 'stock': DSM.s,
                 'inputs': DSM.i,
                 'outputs': DSM.o,
                 'stock change': DSM.i - DSM.o
                 }
        source.data = plot_data
        stdev = source_stdev.data['stdev'][0]
        source_sc.data = {'image': [np.flipud(DSM.s_c)]}
        cohorts_matrix = pd.DataFrame(data=DSM.s_c,  index=DSM.t.as_matrix(), columns=[str(e) for e in plot_data["time"].as_matrix().tolist()])
        cohorts_matrix.insert(0, "time",[str(e) for e in plot_data["time"].as_matrix().tolist()])
        source_bar_temp = ColumnDataSource(data=cohorts_matrix) 
        #temp used only to get the right format
        source_bar.data = source_bar_temp.data 

    def make_bar_plot(source_sc, source_bar):
        # Image plot of the Stock by tie and cohort matrix
        p4 = figure(x_range=(DSM.t[0], DSM.t[len(DSM.t)-1]),
                y_range=(DSM.t[len(DSM.t)-1], DSM.t[0]),
                y_axis_label='Time', x_axis_label='Cohorts',
                plot_width=300, plot_height=250,
                toolbar_location=None)

        p4.image("image", x=DSM.t[0], y=DSM.t[len(DSM.t)-1], 
             dw=len(DSM.t), dh=len(DSM.t),
             source=source_sc, palette="Spectral11")

        p5 = figure(plot_width=600, plot_height=250, x_range=stackers,
                toolbar_location=None)
        p5.vbar_stack(stackers, x='time', width=0.9, 
    #                  legend=["%s cohorts" % x for x in stackers], 
    #                         legend='cohort', 
    #                         fill_color=linear_cmap('cohort', palette=Category20[20], low=cohort[0], high=cohort[-1]),
    #                        fill_color=factor_cmap('cohort', palette=Category20[20],factors=cohort, end=30),
                             color=palette,
                             source=source_bar)
        p5.xaxis.visible = False
        p5.yaxis.axis_label = 'Stock size'
        from bokeh.models import SingleIntervalTicker, LinearAxis

        xaxis = LinearAxis()
        ticker = SingleIntervalTicker(interval=5, num_minor_ticks=10)

        xaxis.ticker=ticker
        xaxis.axis_label = 'Time'

        p5.add_layout(xaxis, 'below')
        return row(p4,p5)

    # Defining sliders for lifetime and selection widget for the input distribution
    lifetime_param = Slider(start=2, end=50, value=15, step=1, 
                            title="Mean lifetime", callback_policy="mouseup")
    stdev_param = Slider(start=0.5, end=15, value=5, step=.1, 
                         title="Standard dev.", callback_policy="mouseup")
    input_selection = Select(title="Stock distribution", value="Constant", 
                             options=data.columns.get_values().tolist()[1:])


    # Fake callbacks, just using JS to enable the mouseup callback_policy in slider
    # This is used because the Bokeh server only enables instant refresh of sliders
    # values, hence generating too many calculations
    # The mouseup policy enables to launch a new calculation only when the user
    # releases the button, but it is necessary to use JS callbacks for this
    lifetime_param.callback = CustomJS(args=dict(source=source_lifetime), code="""
        source.data = { lifetime: [cb_obj.value], stdev: [1] }
    """)
    stdev_param.callback = CustomJS(args=dict(source=source_stdev), code="""
        source.data = { stdev: [cb_obj.value], lifetime: [1] }
    """)

    # Launching the real calbacks every time the false datasources are modified
    source_lifetime.on_change('data', update_lifetime) 
    source_stdev.on_change('data', update_stdev) 
    input_selection.on_change("value", update_input)

    # Generating the plots

    plot_data = {'time': DSM.t,
                 'stock': DSM.s,
                 'inputs': DSM.i,
                 'outputs': DSM.o,
                 'stock change': DSM.i - DSM.o
                 }
    # Hover tool (-> needs to be imporved)
    hover = HoverTool(
            tooltips=[
                ("(x,y)", "(time, $inputs")
            ]
        )

    
    stackers = [str(e) for e in cohorts_matrix["time"].as_matrix().tolist()]
    palette = [Category20[20][i%19] for i in range(len(stackers))]


   

    options = dict(plot_width=300, plot_height=200
    #               ,
    #               tools="pan,wheel_zoom,box_zoom,box_select,lasso_select,hover"
                   )



    # Inflows and outflows
    p1 = figure(title="Inputs and outputs",  **options)
    p1.circle("time", "inputs", color="green", source=source)
    p1.circle("time", "outputs",  line_color="red", fill_color=None, source=source)

    # Stock change
    p2 = figure(title="Stock change", **options)
    p2.circle("time", "stock change",  line_color="black", fill_color=None, source=source)

    # Stock plot
    p3 = figure(title="Stock", tools="hover", **options)
    p3.circle("time", "stock", color="blue", source=source)

    # first line of the plot and title
    p = gridplot([[ p1, p2, p3]], toolbar_location="right", title="MFA Dynamic model")
    #show(p)



    # Bar chart of cohorts vs time


    #
    #p5=Bar(cohorts_matrix_melted, label='time', values='value', agg='sum',
    #                stack='cohort', palette=Category20[20], title="Stacked bars",
    #                plot_width=600, plot_height=250)



    # Putting the title and the widgets in a row container
    widgets =column(Div(text="<h1>Stock Dynamics - Stock Driven Model</h1>", width=500,),
                    row(input_selection, lifetime_param, stdev_param))

    # Adding the plots and widgets to the root of the document
    #curdoc().add_root(column(widgetbox(input_selection), widgetbox(lifetime_param), widgetbox(stdev_param), p, p4))
    layout = column( widgets, p, make_bar_plot(source_sc, source_bar))
    curdoc.add_root(layout)
    curdoc.title = " Dynamic MFA model"


    # Adding the fake datasources to the root
    curdoc.add_root(source_lifetime)
    curdoc.add_root(source_stdev)

    #grid = gridplot([[widgetbox(input_selection), widgetbox(lifetime_param), widgetbox(stdev_param)], [p], [None, p4, None]])
    #curdoc.add_root(grid)

    #curdoc.add_root(layout([[widgetbox(input_selection), widgetbox(lifetime_param), widgetbox(stdev_param)], [p], [None, p4, None]]))
    #
    #cd C:\Users\romainb\Documents\2. Dynamic MFA Course\Visualisation
    #bokeh serve --show inflow_driven_viz.py
    
 # Set up the Application 
handler = FunctionHandler(modify_doc)
app = Application(handler) 


In [4]:
# Show the application
# Make sure the URL matches your Jupyter instance
show(app, notebook_url="localhost:8888", notebook_handle=True)
#show(modify_doc)

Lifetime 21
Lifetime 30
Input distribution Linear growth
